In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from python_module.pricing_model import BSMModel, SABRModel
from python_module.weights_optimization import find_min_abs_weights, fast_find_min_abs_weights

In [2]:
# Market Data Generation Inputs
F = 100
T = 1
alpha = 0.2
beta = 1
rho = -0.9
nu = 0.3
r = 0

In [3]:
# Generate options
results = dict()
for K in range(60, 141, 1):
    option_type = 'call' if K > F else 'put'
    out = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type=option_type)
    out['option_type'] = option_type
    out['delta_call'] = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type='call')['delta']
    results[K] = out

In [4]:
# Generate Weights
df = pd.DataFrame(results)
df = df.transpose()
df['weight'] = (1 / np.power(df.index, 2)) * 1_000_000

In [5]:
# Compute COG
X = df[['gamma', 'vanna', 'volga']]
y = df['theta']
model = LinearRegression(fit_intercept=False)
model.fit(X.values, y.values)
y_pred = model.predict(X.values)
coef = model.coef_
r2 = r2_score(y.values, y_pred)
cogwa = (X * (coef))
cogwa.columns = ['costof_gamma', 'costof_vanna', 'costof_volga']
df = pd.concat([df, cogwa], axis=1)
df['costof_other'] = df['theta'] - df[['costof_gamma', 'costof_vanna', 'costof_volga']].sum(axis=1)

In [6]:
# Delta Bucketing
bins = [0, 0.25, 0.5, 0.75, 1]
labels = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1']
df['delta_bucket'] = pd.cut(df['delta_call'], bins=bins, labels=labels)

In [7]:
# Compute adjusted ref risk for portfolio reduction
risk_ref = 'theta'
df[f'pos_{risk_ref}'] = df[risk_ref] * df['weight']

In [8]:
# Compute weight_reduction with discrete approach
df['risk_bucket'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)[f'pos_{risk_ref}'].sum().to_dict()))
df['qty_to_trade'] = df['risk_bucket'] / df[risk_ref]
df['min_qty_to_trade'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)['qty_to_trade'].min().to_dict())) 
df['weight_reduction'] = (df['qty_to_trade']==df['min_qty_to_trade']) * df['min_qty_to_trade']
df['weight_reduction'] = df['weight_reduction'].replace(0, np.nan)

In [9]:
# Compute weight_reduction with optimized approach
index_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga', 'costof_other']].multiply(df['weight'], axis=0).sum()
weights = fast_find_min_abs_weights(df, index_cog, combination_size=4)
df.loc[weights.index, 'weight_opt'] = weights.values
opt_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga', 'costof_other']].multiply(df['weight_opt'], axis=0).sum()

In [10]:
px.scatter(df[['weight', 'weight_reduction', 'weight_opt']])

In [11]:
df

,IV,price,delta,gamma,vega,theta,vanna,volga,option_type,delta_call,...,costof_vanna,costof_volga,costof_other,delta_bucket,pos_theta,risk_bucket,qty_to_trade,min_qty_to_trade,weight_reduction,weight_opt
60,0.26088,0.190562,-0.018375,0.001727,0.045054,-0.002332,-0.31563,65.920132,put,0.981625,...,-0.000673,-0.000231,-0.000061,0.75-1,-0.647794,-36.147061,15500.066294,2739.670480,NaN,NaN
61,0.258927,0.21695,-0.020751,0.001929,0.049954,-0.002566,-0.343327,69.986695,put,0.979249,...,-0.000732,-0.000245,-0.000062,0.75-1,-0.689701,-36.147061,14084.858289,2739.670480,NaN,NaN
62,0.256999,0.246412,-0.023374,0.002149,0.055236,-0.002817,-0.37216,74.006661,put,0.976626,...,-0.000793,-0.000259,-0.000062,0.75-1,-0.732717,-36.147061,12833.748831,2739.670480,NaN,NaN
63,0.255095,0.279231,-0.026264,0.002388,0.060909,-0.003083,-0.402015,77.941864,put,0.973736,...,-0.000857,-0.000273,-0.000062,0.75-1,-0.776734,-36.147061,11725.178164,2739.670480,NaN,NaN
64,0.253214,0.315711,-0.02944,0.002645,0.066986,-0.003365,-0.432757,81.751983,put,0.97056,...,-0.000922,-0.000286,-0.000062,0.75-1,-0.821635,-36.147061,10740.740603,2739.670480,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0.15499,0.159837,0.028299,0.004182,0.06482,-0.001993,0.862125,164.356283,call,0.028299,...,0.001838,-0.000575,0.000056,0-0.25,-0.107772,-9.232552,4631.679085,834.185425,NaN,NaN
137,0.153985,0.135433,0.024567,0.00374,0.057595,-0.00176,0.793467,156.108876,call,0.024567,...,0.001691,-0.000546,0.000057,0-0.25,-0.093754,-9.232552,5246.73472,834.185425,NaN,NaN
138,0.15299,0.114308,0.021241,0.00333,0.050952,-0.001547,0.726613,147.412162,call,0.021241,...,0.001549,-0.000516,0.000058,0-0.25,-0.081215,-9.232552,5969.333498,834.185425,NaN,NaN
139,0.152005,0.096096,0.018291,0.002952,0.044877,-0.001353,0.662035,138.392003,call,0.018291,...,0.001411,-0.000484,0.000058,0-0.25,-0.070052,-9.232552,6821.37639,834.185425,NaN,NaN
